# =======================================================
## STREAMLINED VA-COUNT EEM Module
# =======================================================


In [ ]:
# # === 1. GO TO A CLEAN DIRECTORY & CLEAN UP ===
# print("Cleaning up old directories...")
# %cd /content/
# !rm -rf VA-Count

# # === 2. INSTALL ALL BUILD TOOLS & DEPENDENCIES ===
# print("Installing build tools and dependencies...")
# !pip install --upgrade pip setuptools wheel
# !pip install ninja cython pycocotools
# !sudo apt-get install -y build-essential
# !pip install torch torchvision
# !pip install timm transformers
# !pip install git+https://github.com/openai/CLIP.git




In [ ]:
# # === 3. CLONE REPOS IN THE CORRECT STRUCTURE ===
# print("Cloning repositories...")
# !git clone https://github.com/HopooLinZ/VA-Count.git
# %cd VA-Count
# !git clone https://github.com/IDEA-Research/GroundingDINO.git

# # === 4. GroundingDINO FIX  ===
# # This manually patches a known bug in the C++/CUDA code
# print("Applying manual patch to GroundingDINO CUDA files...")
# %cd GroundingDINO/groundingdino/models/GroundingDINO/csrc/MsDeformAttn
# !sed -i 's/value.type()/value.scalar_type()/g' ms_deform_attn_cuda.cu
# !sed -i 's/value.scalar_type().is_cuda()/value.is_cuda()/g' ms_deform_attn_cuda.cu
# print("Patch applied!")

# # === 5. RUN THE INSTALLATION ===
# print("Building GroundingDINO...")
# %cd /content/VA-Count/GroundingDINO

# # We use --no-build-isolation to force it to use the packages
# # we already installed, preventing the "chicken-and-egg" error
# !pip install --no-build-isolation -e .

# print("✅ Setup complete!")

In [ ]:
import sys

# ==== 0) Mount Drive (optional) ====
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Base project dir (change if you like)
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2"
!mkdir -p "$PROJECT_DIR"
%cd "$PROJECT_DIR"

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count"

!pwd

/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count
/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count


In [ ]:


# Path to your script on Google Drive
sys.path.append("/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count")

# Path to the GroundingDINO we just built in /content/
sys.path.append("/content/VA-Count/GroundingDINO")

# %load_ext autoreload
# %autoreload 2

In [ ]:
# ==== 2) Core deps (PyTorch + utils) ====
# If your Colab runtime has CUDA 12.1, this is fine. Otherwise remove the --index-url line.
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install opencv-python matplotlib tqdm pandas scikit-image h5py addict yapf supervision
# CLIP for GroundingDINO
!pip -q install git+https://github.com/openai/CLIP.git

!pip install timm==0.4.9
!pip install imgaug

# --- 3. Install Dependencies ---
# Install VA-Count requirements
print("\nInstalling requirements from VA-Count/requirements.txt...")
# Use relative path since we are in project_dir
get_ipython().system('pip install -r requirements.txt')

# Install PyTorch
print("\nInstalling PyTorch (with CUDA 12.1 support)...")
get_ipython().system('pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121')

# Install other dependencies
print("\nInstalling other dependencies (supervision, segment-anything, timm, einops, etc.)...")
# Added 'timm' and 'einops' proactively to resolve potential missing dependencies for GroundingDINO
get_ipython().system('pip install supervision segment-anything ftfy==6.1.1 transformers timm einops')

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

Installing requirements from VA-Count/requirements.txt...
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
ERROR: Could not find a version that satisfies the requirement torch==1.13.1+cu116 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0)
ERROR: No matching distribution found for torch==1.13.1+cu116

Installing PyTorch (with CUDA 12.1 support)...
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121

Installing other dependencies (supervision, segment-anything, timm, einops, etc.)...


In [ ]:
# !mkdir -p VA-Count/weights
# !wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth -P VA-Count/weights/

In [ ]:
# %cd "/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count"

# !pwd

In [ ]:
# !python grounding_pos_dummy.py --root_path ./data/FSC147/

In [ ]:
# import os
# import json
# from PIL import Image

# def create_patches_from_json(json_path, image_dir, output_dir):
#     """
#     Crops patches from images based on coordinates in a JSON file
#     and saves them to an output directory.
#     """

#     # 1. Ensure the output directory exists
#     os.makedirs(output_dir, exist_ok=True)
#     print(f"Output directory created at: {output_dir}")

#     # 2. Read the JSON annotation file
#     try:
#         with open(json_path, 'r') as f:
#             data = json.load(f)
#     except FileNotFoundError:
#         print(f"Error: JSON file not found at {json_path}")
#         return
#     except json.JSONDecodeError:
#         print(f"Error: Could not decode JSON file. Is it empty or corrupt?")
#         return

#     print(f"Successfully loaded {json_path}")

#     patch_count = 0
#     image_count = 0

#     # 3. Loop through every image in the JSON
#     for image_name, info in data.items():

#         coordinates_list = info.get("box_examples_coordinates")

#         # Skip if this image has no coordinate data
#         if not coordinates_list:
#             continue

#         # 4. Load the original image
#         image_path = os.path.join(image_dir, image_name)

#         if not os.path.exists(image_path):
#             print(f"Warning: Source image not found, skipping: {image_path}")
#             continue

#         try:
#             img = Image.open(image_path).convert("RGB")
#             image_count += 1
#         except Exception as e:
#             print(f"Error loading image {image_path}: {e}")
#             continue

#         # 5. Loop through the 3 box coordinates for this image
#         for i, box_coords in enumerate(coordinates_list):
#             try:
#                 # Extract the [x1, y1] and [x2, y2] coordinates
#                 # Based on grounding_pos.py, the format is [[x1, y1], [x1, y2], [x2, y2], [x2, y1]]
#                 x1 = int(box_coords[0][0])
#                 y1 = int(box_coords[0][1])
#                 x2 = int(box_coords[2][0])
#                 y2 = int(box_coords[2][1])

#                 # 6. Crop the image
#                 # Ensure coordinates are valid and within image bounds
#                 crop_x1 = max(0, x1)
#                 crop_y1 = max(0, y1)
#                 crop_x2 = min(img.width, x2)
#                 crop_y2 = min(img.height, y2)

#                 # Skip if the box is invalid (e.g., zero size)
#                 if crop_x1 >= crop_x2 or crop_y1 >= crop_y2:
#                     print(f"Warning: Skipping invalid box for {image_name}")
#                     continue

#                 cropped_img = img.crop((crop_x1, crop_y1, crop_x2, crop_y2))

#                 # 7. Save the new cropped patch
#                 base_name, _ = os.path.splitext(image_name)
#                 output_filename = f"{base_name}_patch_{i}.jpg"
#                 output_path = os.path.join(output_dir, output_filename)

#                 cropped_img.save(output_path, "JPEG")
#                 patch_count += 1

#             except Exception as e:
#                 print(f"Error processing box {i} for {image_name}: {e}")

#     print("\n--- Cropping Complete ---")
#     print(f"Processed {image_count} images.")
#     print(f"Saved {patch_count} patches to {output_dir}")

# # --- Main execution ---
# if __name__ == "__main__":

#     # Define the paths based on your project structure
#     JSON_FILE_PATH = './data/FSC147/annotation_FSC147_pos.json'
#     SOURCE_IMAGE_DIR = './data/FSC147/images_384_VarV2'
#     PATCH_OUTPUT_DIR = './data/FSC147/box'

#     # Run the function
#     create_patches_from_json(JSON_FILE_PATH, SOURCE_IMAGE_DIR, PATCH_OUTPUT_DIR)

In [ ]:
# !python datasetmake.py

In [ ]:
# import json

# json_file_path = './data/FSC147/Train_Test_Val_FSC_147.json'

# try:
#     # 1. Read the JSON file
#     with open(json_file_path, 'r') as f:
#         data = json.load(f)
#     print(f"Successfully loaded {json_file_path}")

#     # 2. Process all three splits: train, val, and test
#     splits = ['train', 'val', 'test']

#     for split in splits:
#         filenames = data.get(split)

#         if not filenames:
#             print(f"Warning: No '{split}' key found in JSON file. Skipping...")
#             continue

#         # Define the output path for the .txt file
#         output_txt_path = f'./data/FSC147/{split}.txt'

#         # 3. Write the correct filenames to the corresponding .txt file
#         try:
#             with open(output_txt_path, 'w') as f:
#                 for filename in filenames:
#                     f.write(f"{filename}\n")

#             print(f"Successfully created '{output_txt_path}' with {len(filenames)} filenames.")

#         except Exception as e:
#             print(f"Error writing to {output_txt_path}: {e}")

# except FileNotFoundError:
#     print(f"Error: JSON file not found at {json_file_path}")
# except Exception as e:
#     print(f"An error occurred: {e}")

In [ ]:
# !pwd

# !mkdir -p ./data/out/classify

# !python biclassify.py

In [ ]:
#!python grounding_pos.py --root_path ./data/FSC147/

In [ ]:
#!python grounding_neg.py --root_path ./data/FSC147/

In [ ]:
#combine annotatiaon_FSC147_neg.json' and pos into annotation_FSC147_negative1.json'
import json
import os

%cd "/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count/data/FSC147"

# --- Configuration ---
FILE_NEG = 'annotation_FSC147_neg.json'
FILE_POS = 'annotation_FSC147_pos.json' # Assuming this is the "pos" file
FILE_OUTPUT = 'annotation_FSC147_negative1.json'
# ---------------------

print(f"Starting JSON combination...")
print(f"Working directory: {os.getcwd()}")

combined_data = {}
data_neg = {}
data_pos = {}

# --- 1. Load Negative Annotations ---
try:
    with open(FILE_NEG, 'r') as f:
        data_neg = json.load(f)
    print(f"Successfully loaded {len(data_neg)} entries from {FILE_NEG}")
except FileNotFoundError:
    print(f"ERROR: Cannot find file: {FILE_NEG}")
except json.JSONDecodeError:
    print(f"ERROR: Could not decode JSON from {FILE_NEG}. Check file format.")

# --- 2. Load Positive Annotations ---
try:
    with open(FILE_POS, 'r') as f:
        data_pos = json.load(f)
    print(f"Successfully loaded {len(data_pos)} entries from {FILE_POS}")
except FileNotFoundError:
    print(f"ERROR: Cannot find file: {FILE_POS}")
except json.JSONDecodeError:
    print(f"ERROR: Could not decode JSON from {FILE_POS}. Check file format.")

# --- 3. Combine Data (assuming both are dictionaries) ---
if isinstance(data_neg, dict) and isinstance(data_pos, dict):
    # Create a copy and update it.
    # This merges the two dictionaries.
    # If any keys are in both files, the value from data_pos will be used.
    combined_data = data_neg.copy()
    combined_data.update(data_pos)

    print(f"Total combined entries: {len(combined_data)}")

    # --- 4. Write to Output File ---
    try:
        with open(FILE_OUTPUT, 'w') as f:
            json.dump(combined_data, f, indent=4)
        print(f"✅ Successfully combined annotations and saved to {FILE_OUTPUT}")
    except IOError as e:
        print(f"ERROR: Could not write to output file {FILE_OUTPUT}. {e}")
else:
    if not combined_data:
        print("ERROR: Could not load any data, output file will not be created.")
    else:
        print("ERROR: Both JSON files must contain a dictionary (key-value) structure to be merged.")


/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count/data/FSC147
Starting JSON combination...
Working directory: /content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count/data/FSC147
Successfully loaded 6146 entries from annotation_FSC147_neg.json
Successfully loaded 6146 entries from annotation_FSC147_pos.json
Total combined entries: 6146
✅ Successfully combined annotations and saved to annotation_FSC147_negative1.json


# =======================================================
## STREAMLINED VA-COUNT NSM Module
# =======================================================

In [ ]:

import sys
import os

# --- 1. Mount Drive and Define Paths ---
print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define absolute paths
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2"
VA_COUNT_DIR = os.path.join(PROJECT_DIR, "VA-Count")

# --- 2. Move to Project Directory & Clone VA-Count ---
print(f"Changing directory to: {PROJECT_DIR}")
os.makedirs(PROJECT_DIR, exist_ok=True)
%cd "$PROJECT_DIR"

if not os.path.exists(VA_COUNT_DIR):
    print("Cloning VA-Count...")
    !git clone https://github.com/HopooLinZ/VA-Count.git
else:
    print("VA-Count directory found. Skipping clone.")

# Move into the VA-Count root directory
%cd "$VA_COUNT_DIR"
print(f"Current Working Directory: {os.getcwd()}")


# --- 3. Install All Dependencies (The Clean Way) ---

# 3a. Install a modern PyTorch version compatible with Colab (CUDA 12.1)
# This overrides the old 'cu116' version in requirements.txt
print("\nInstalling PyTorch for Colab...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 3b. Install all requirements from the file
# We use --no-deps to prevent it from trying to downgrade our new PyTorch
print("\nInstalling dependencies from requirements.txt...")
# We tell pip to ignore the torch/torchvision requirements in the file
# since we just installed a working version.
!pip install -r requirements.txt

# 3c. Install remaining dependencies that were NOT in requirements.txt
print("\nInstalling additional dependencies (OpenCV, CLIP, Transformers, etc.)...")
!pip install -q opencv-python scikit-image h5py addict yapf supervision
!pip install -q segment-anything ftfy==6.1.1 transformers einops
!pip install -q git+https://github.com/openai/CLIP.git

# # --- 4. Build GroundingDINO (from your commented-out steps) ---

# # 4a. Install build tools
# print("\nInstalling GroundingDINO build tools...")
# !pip install -q ninja cython pycocotools
# !sudo apt-get install -y build-essential

# # 4b. Clone GroundingDINO (inside VA-Count) if it's not already there
GROUNDING_DINO_DIR = os.path.join(VA_COUNT_DIR, "GroundingDINO")
# if not os.path.exists(GROUNDING_DINO_DIR):
#     print("Cloning GroundingDINO...")
#     !git clone https://github.com/IDEA-Research/GroundingDINO.git
# else:
#     print("GroundingDINO directory found. Skipping clone.")

# # 4c. Apply manual C++/CUDA patch
# print("\nApplying manual patch to GroundingDINO CUDA files...")
# PATCH_DIR = os.path.join(GROUNDING_DINO_DIR, "groundingdino/models/GroundingDINO/csrc/MsDeformAttn")
# %cd "$PATCH_DIR"
# !sed -i 's/value.type()/value.scalar_type()/g' ms_deform_attn_cuda.cu
# !sed -i 's/value.scalar_type().is_cuda()/value.is_cuda()/g' ms_deform_attn_cuda.cu
# print("Patch applied!")

# # 4d. Build GroundingDINO from its root directory
# print("\nBuilding GroundingDINO...")
# %cd "$GROUNDING_DINO_DIR"
# !pip install --no-build-isolation -e .


!pip install imgaug


# This downgrades NumPy (for imgaug) and timm (for models_mae_noct).
print("\n[FIX] Forcing NumPy 1.26.4 (for imgaug)...")
!pip install numpy==1.26.4
print("\n[FIX] Forcing timm 0.4.9 (for models_mae_noct)...")
!pip install timm==0.4.9

# --- 5. Final Setup: Paths and Weights ---
print("\nFinalizing setup...")
# Go back to the VA-Count root
%cd "$VA_COUNT_DIR"
!pwd

# Add the project root and GroundingDINO to sys.path
sys.path.append(os.getcwd())
sys.path.append(GROUNDING_DINO_DIR)

# Download weights (if not present)
print("Downloading GroundingDINO weights...")
WEIGHTS_DIR = "weights"
!mkdir -p "$WEIGHTS_DIR"
WEIGHTS_FILE = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(WEIGHTS_DIR, WEIGHTS_FILE)

if not os.path.exists(WEIGHTS_PATH):
    !wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth -P "$WEIGHTS_DIR"
else:
    print("Weights already present.")

print("\n✅ Setup complete!")
print("You can now run: !python grounding_pos_dummy.py --root_path ./data/FSC147/")


Mounting Google Drive...
Mounted at /content/drive
Changing directory to: /content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2
/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2
VA-Count directory found. Skipping clone.
/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count
Current Working Directory: /content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count

Installing PyTorch for Colab...

Installing dependencies from requirements.txt...
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
ERROR: Ignored the following yanked versions: 1.11.0, 1.14.0rc1
ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python <3.12,>=3.8; 1.10.0rc1 Requires-Python <3.12,>=3.8; 1.10.0rc2 Requires-Python <3.12,>=3.8; 1.10.1 Requires-Python <3.12,>=3.8; 1.21.2 Requires-Python >=3.


[FIX] Forcing timm 0.4.9 (for models_mae_noct)...
  Using cached timm-0.4.9-py3-none-any.whl.metadata (27 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.1/346.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 1.0.22
    Uninstalling timm-1.0.22:
      Successfully uninstalled timm-1.0.22

Finalizing setup...
/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count
/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count
Weights already present.

✅ Setup complete!
You can now run: !python grounding_pos_dummy.py --root_path ./data/FSC147/


In [ ]:
import sys, os

# --- 1. Mount Drive and Define Paths ---
print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define absolute paths
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2"
VA_COUNT_DIR = os.path.join(PROJECT_DIR, "VA-Count")
GROUNDING_DINO_DIR = os.path.join(VA_COUNT_DIR, "GroundingDINO")

# Ensure paths are set (just in case)
if VA_COUNT_DIR not in sys.path:
    sys.path.append(VA_COUNT_DIR)
if GROUNDING_DINO_DIR not in sys.path:
    sys.path.append(GROUNDING_DINO_DIR)

# Change to the correct directory to run the script
%cd "$VA_COUNT_DIR"

# # 5b. Download MAE weights (The fix for your error)
# print("Downloading MAE weights...")
# WEIGHTS_DIR = "weights"
# MAE_WEIGHTS_FILE = "mae_pretrain_vit_base_full.pth"
# MAE_WEIGHTS_PATH = os.path.join(WEIGHTS_DIR, MAE_WEIGHTS_FILE)
# if not os.path.exists(MAE_WEIGHTS_PATH):
#     !wget -q https://dl.fbaipublicfiles.com/mae/pretrain/mae_pretrain_vit_base_full.pth -P "$WEIGHTS_DIR"
# else:
#     print("MAE weights already present.")



Mounting Google Drive...
Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count


In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count


In [ ]:
# #helper to unzip density maps form uploaded density maps zip file

# destination_dir = os.path.join(PROJECT_DIR, "VA-Count/data/FSC147")
# zip_file_path = os.path.join(PROJECT_DIR, "VA-Count/data/FSC147/gt_density_map_adaptive_384_VarV2.zip")
# os.makedirs(destination_dir, exist_ok=True)

# !unzip -n "$zip_file_path" -d "$destination_dir"

In [ ]:
# Change to the correct directory to run the script
%cd "$VA_COUNT_DIR"

# Now, run your script
!python FSC_pretrain.py \
    --epochs 200 \
    --warmup_epochs 2 \
    --blr 1.5e-4 \
    --weight_decay 0.05 \
    --resume "./data/out/pre_4_dir/checkpoint__pretraining_90.pth"

/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count
2025-11-21 17:23:21.151206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763745801.170482    3942 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763745801.177133    3942 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763745801.192323    3942 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763745801.192349    3942 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:0

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python FSC_train.py \
  --epochs 500 \
  --batch_size 8 \
  --lr 1e-5 \
  --num_workers 4 \
  --output_dir "./data/out/" \
  --resume "./data/out/pre_4_dir/checkpoint__pretraining_90.pth"

>>> [DEBUG] Running FSC_train.py from: /content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count/FSC_train.py
[DEBUG] args in FSC_train.py: Namespace(batch_size=8, epochs=500, accum_iter=1, model='mae_vit_base_patch16', mask_ratio=0.5, norm_pix_loss=False, weight_decay=0.05, lr=1e-05, blr=0.001, min_lr=0.0, warmup_epochs=10, data_path='./data/FSC147/', anno_file=PosixPath('data/FSC147/annotation_FSC147_pos.json'), anno_file_negative='./data/FSC147/annotation_FSC147_neg.json', data_split_file=PosixPath('data/FSC147/Train_Test_Val_FSC_147.json'), class_file=PosixPath('data/FSC147/ImageClasses_FSC147.txt'), im_dir=PosixPath('data/FSC147/images_384_VarV2'), output_dir='./data/out/', device='cuda', seed=0, resume='./data/out/pre_4_dir/checkpoint__pretraining_90.pth', do_resume=False, start_epoch=0, num_workers=4, pin_mem=True, do_aug=True, world_size=1, local_rank=-1, dist_on_itp=False, dist_url='env://', title='finetuning', wandb='AdamW optimized fine-tuning', team='wsen

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python FSC_train.py \
    --epochs 500 \
    --batch_size 8 \
    --lr 1e-5 \
    --num_workers 8 \
    --output_dir "./data/out/" \
    --resume "./data/out/checkpoint__finetuning_last.pth" \
    --do_resume

>>> [DEBUG] Running FSC_train.py from: /content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count/FSC_train.py
[DEBUG] args in FSC_train.py: Namespace(batch_size=8, epochs=500, accum_iter=1, model='mae_vit_base_patch16', mask_ratio=0.5, norm_pix_loss=False, weight_decay=0.05, lr=1e-05, blr=0.001, min_lr=0.0, warmup_epochs=10, data_path='./data/FSC147/', anno_file=PosixPath('data/FSC147/annotation_FSC147_pos.json'), anno_file_negative='./data/FSC147/annotation_FSC147_neg.json', data_split_file=PosixPath('data/FSC147/Train_Test_Val_FSC_147.json'), class_file=PosixPath('data/FSC147/ImageClasses_FSC147.txt'), im_dir=PosixPath('data/FSC147/images_384_VarV2'), output_dir='./data/out/', device='cuda', seed=0, resume='./data/out/checkpoint__finetuning_last.pth', do_resume=True, start_epoch=0, num_workers=8, pin_mem=True, do_aug=True, world_size=1, local_rank=-1, dist_on_itp=False, dist_url='env://', title='finetuning', wandb='AdamW optimized fine-tuning', team='wsense', wandb

In [ ]:
!python FSC_test.py --output_dir ./data/out/results_base --resume ./data/out/fim6_dir/checkpoint__FSC.pth

Not using distributed mode
[04:39:13.892914] job dir: /content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count
[04:39:13.893026] Namespace(model='mae_vit_base_patch16',
mask_ratio=0.5,
norm_pix_loss=False,
data_path='./data/FSC147/',
anno_file='annotation_FSC147_pos.json',
anno_file_negative='./data/FSC147/annotation_FSC147_neg.json',
data_split_file='Train_Test_Val_FSC_147.json',
im_dir='images_384_VarV2',
output_dir='./data/out/results_base',
device='cuda',
seed=0,
resume='./data/out/fim6_dir/checkpoint__FSC.pth',
external=False,
box_bound=-1,
split='test',
num_workers=0,
pin_mem=True,
normalization=True,
world_size=1,
local_rank=-1,
dist_on_itp=False,
dist_url='env://',
distributed=False)
[04:39:18.392838] Resume checkpoint ./data/out/fim6_dir/checkpoint__FSC.pth (99)
[04:39:18.393552] Start testing.
Validation:   0% 0/1190 [00:00<?, ?it/s]/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count/FSC_test.py:303: FutureWarning: `torch.cuda.amp.

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python FSC_test_save.py \
    --resume "./data/out/checkpoint__finetuning_last.pth" \
    --data_path "./data/FSC147/" \
    --anno_file "annotation_FSC147_pos.json" \
    --anno_file_negative "./data/FSC147/annotation_FSC147_neg.json" \
    --data_split_file "Train_Test_Val_FSC_147.json" \
    --im_dir "images_384_VarV2" \
    --split "test" \
    --output_dir "./data/out/results_base"


Not using distributed mode
[07:42:58.610225] job dir: /content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count
[07:42:58.610377] Namespace(model='mae_vit_base_patch16',
mask_ratio=0.5,
norm_pix_loss=False,
data_path='./data/FSC147/',
anno_file='annotation_FSC147_pos.json',
anno_file_negative='./data/FSC147/annotation_FSC147_neg.json',
data_split_file='Train_Test_Val_FSC_147.json',
im_dir='images_384_VarV2',
output_dir='./data/out/results_base',
device='cuda',
seed=0,
resume='./data/out/checkpoint__finetuning_last.pth',
external=False,
box_bound=-1,
split='test',
num_workers=0,
pin_mem=True,
normalization=True,
world_size=1,
local_rank=-1,
dist_on_itp=False,
dist_url='env://',
distributed=False)
[07:43:16.191945] Resume checkpoint ./data/out/checkpoint__finetuning_last.pth (127)
[07:43:16.212468] Start testing.
Validation:   0% 0/1190 [00:00<?, ?it/s]/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count/FSC_test_save.py:274: FutureWarning: `tor

In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/COMP 6970/VA-Count-Project_2/VA-Count
